In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import os
from ccfj import GetStationPairs
from geopy.distance import great_circle
from scipy.interpolate import interp1d
from scipy.interpolate import griddata
#import shapefile
import geopandas as gp
import yaml
import math
#from pykml import parser


In [10]:
import sys
sys.path.append(r'../')
from toollib_standard import plotlib

In [11]:
flag_project = 0 # 0--regular; 1--repartrition

In [12]:
if flag_project == 0:
    with open('a-project.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader)
    name_project = proj['name']
    #name_project = 'project/output_FJSJ_16-01/'               # Harmon server
elif flag_project == 1:
    with open('a-project_repar.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader)
    name_project = proj['name']
    #name_project = 'project_repartrition/repartrition_01-03/'               # Harmon server


In [13]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  ./
dir_project_workspace:  ./
dir_project:  ./project/output_FJSJ_15-01/


In [14]:
file_xlsx = 'structure_model.xlsx'
#file_xlsx = 'structure_model_fund.xlsx'

In [15]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary


In [16]:
dir_image = dir_project + info_basic['dir_image']
dir_inv_dispernet = dir_project + info_basic['dir_inv_dispernet']
key_subworks = info_basic['key_subworks']
M = len(key_subworks)
key_subworks

['08-03',
 '09-03',
 '10-03',
 '11-03',
 '12-03',
 '13-03',
 '14-03',
 '15-03',
 '16-03',
 '17-03',
 '18-03',
 '19-03',
 '20-03',
 '21-03',
 '22-03',
 '23-03',
 '24-03',
 '25-03',
 '26-03',
 '27-03',
 '28-03',
 '29-03',
 '30-03',
 '31-03',
 '32-03',
 '33-03',
 '34-03',
 '35-03',
 '36-03',
 '37-03',
 '38-03',
 '39-03',
 '40-03',
 '41-03',
 '42-03',
 '43-03',
 '44-03',
 '45-03',
 '46-03',
 '47-03',
 '48-03',
 '49-03',
 '50-03',
 '51-03',
 '52-03',
 '53-03',
 '08-04',
 '09-04',
 '10-04',
 '11-04',
 '12-04',
 '13-04',
 '14-04',
 '15-04',
 '16-04',
 '17-04',
 '18-04',
 '19-04',
 '20-04',
 '21-04',
 '22-04',
 '23-04',
 '24-04',
 '25-04',
 '26-04',
 '27-04',
 '28-04',
 '29-04',
 '30-04',
 '31-04',
 '32-04',
 '33-04',
 '34-04',
 '35-04',
 '36-04',
 '37-04',
 '38-04',
 '39-04',
 '40-04',
 '41-04',
 '42-04',
 '43-04',
 '44-04',
 '45-04',
 '46-04',
 '47-04',
 '48-04',
 '49-04',
 '50-04',
 '51-04',
 '52-04',
 '53-04',
 '08-05',
 '09-05',
 '10-05',
 '11-05',
 '12-05',
 '13-05',
 '14-05',
 '15-05',


In [17]:
struc = {}
flag = 0
file_struc = dir_inv_dispernet + file_xlsx
df = pd.read_excel(file_struc,sheet_name=None)
for key_subwork in key_subworks:
    struc[key_subwork] = df[key_subwork]
    flag += 1
    print('Read structure model: %s (%d/%d)' % (key_subwork, flag, M))


Read structure model: 08-03 (1/736)
Read structure model: 09-03 (2/736)
Read structure model: 10-03 (3/736)
Read structure model: 11-03 (4/736)
Read structure model: 12-03 (5/736)
Read structure model: 13-03 (6/736)
Read structure model: 14-03 (7/736)
Read structure model: 15-03 (8/736)
Read structure model: 16-03 (9/736)
Read structure model: 17-03 (10/736)
Read structure model: 18-03 (11/736)
Read structure model: 19-03 (12/736)
Read structure model: 20-03 (13/736)
Read structure model: 21-03 (14/736)
Read structure model: 22-03 (15/736)
Read structure model: 23-03 (16/736)
Read structure model: 24-03 (17/736)
Read structure model: 25-03 (18/736)
Read structure model: 26-03 (19/736)
Read structure model: 27-03 (20/736)
Read structure model: 28-03 (21/736)
Read structure model: 29-03 (22/736)
Read structure model: 30-03 (23/736)
Read structure model: 31-03 (24/736)
Read structure model: 32-03 (25/736)
Read structure model: 33-03 (26/736)
Read structure model: 34-03 (27/736)
Read struc

In [18]:
"""
struc = {}
flag = 0
for key_subwork in key_subworks:
    file_struc = dir_inv_dispernet + '/model_'+key_subwork+'.txt'
    #struc[key_subwork] = np.loadtxt(file_struc)
    file = np.loadtxt(file_struc)

    flag += 1
    print('Read structure model: %s (%d/%d)' % (key_subwork, flag, M))
"""

"\nstruc = {}\nflag = 0\nfor key_subwork in key_subworks:\n    file_struc = dir_inv_dispernet + '/model_'+key_subwork+'.txt'\n    #struc[key_subwork] = np.loadtxt(file_struc)\n    file = np.loadtxt(file_struc)\n\n    flag += 1\n    print('Read structure model: %s (%d/%d)' % (key_subwork, flag, M))\n"

In [19]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist() 
lat_all =  stainfo['latitude'].tolist() 
lon_all =  stainfo['longitude'].tolist() 

In [20]:
stalistname = dir_project+ info_basic['stalistname']
stainfo_all = pd.read_excel(stalistname,sheet_name=None)

In [21]:
num_stations = []
for key_subwork in key_subworks:
    stainfo = stainfo_all[key_subwork]
    stalist = stainfo['Station'].tolist()
    num_stations.append(len(stalist))

In [22]:
locinfo = pd.read_excel(dir_project + 'subwork_location.xlsx',sheet_name='location')

In [23]:
#key_subworks = info_basic['key_subworks']
lat_this = []
lon_this = []
for key_subwork in key_subworks:
    lon_this.append(locinfo['lon_centroid'][key_subworks.index(key_subwork)])
    lat_this.append(locinfo['lat_centroid'][key_subworks.index(key_subwork)])

In [24]:
faults = np.load('clark_faults.npy', allow_pickle='TRUE').item()

### Plot horizontal of certain depth

In [41]:
#%%capture
M = 100
dz = 4
N = 50


d = -27.3

for i in range(N):
#for i in range(40,41):
    if i*dz < 120:
        c_min = 0.6
        c_max = 1
    else:
        c_min = 0.8
        c_max = 1.6 
    #vs,yy,xx,xx_0,yy_0,vs_all = extract_horizontal(M,i,key_subworks,d)
    fig,ax = plt.subplots(1,1,figsize=(8,6))
    ax.scatter(lon_all, lat_all,marker='.',color='k')
    #ax.set_xticks([])  #去掉横坐标值
    #ax.set_yticks([])  #去掉纵坐标值
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Vs at depth '+str((i+1)*dz))
    flag_add = 0.0005
    ax.set_xlim([min(lon_all)-flag_add, max(lon_all)+flag_add])
    ax.set_ylim([min(lat_all)-flag_add, max(lat_all)+flag_add])
    for j in range(len(faults)):
        ax.plot(faults['clark'+str(j+1)]['lon'], faults['clark'+str(j+1)]['lat'], 'k')
    vs_all = []
    for key_subwork in key_subworks:
        vs_all.append(struc[key_subwork]['vs'].tolist()[i])
    im = ax.scatter(lon_this,lat_this,c=vs_all,cmap='gist_rainbow')
    plt.colorbar(im)
    ax.set_xticks([])  #去掉横坐标值
    ax.set_yticks([])  #去掉纵坐标值
    """
    ax[1].scatter(lon_all, lat_all,marker='.',color='k')
    
    ax[1].set_xlabel('Longitude')
    ax[1].set_ylabel('Latitude')
    ax[1].set_title('Vs at depth '+str((i+1)*dz))
    flag_add = 0.0005
    ax[1].set_xlim([min(lon_all)-flag_add, max(lon_all)+flag_add])
    ax[1].set_ylim([min(lat_all)-flag_add, max(lat_all)+flag_add])
    vs_all = []
    for key_subwork in key_subworks:
        vs_all.append(struc[key_subwork]['vs'].tolist()[i+1])
    im = ax[1].scatter(lon_this,lat_this,c=vs_all, vmin = c_min, vmax=c_max,cmap='gist_rainbow')
    plt.colorbar(im)
    """

    # Vs plot
    #cbar = np.linspace(c_min,c_max,20)
    #im = ax.imshow(vs, vmin= c_min, vmax=c_max, origin='lower', extent=[min(lon_all), max(lon_all), min(lat_all), max(lat_all)], aspect='auto',cmap='gist_rainbow')
    #im = ax.imshow(vs, origin='lower', aspect='auto',cmap='gist_rainbow')
    #im = ax.contourf(xi,yi,zi,15,cmap=plt.cm.jet,vmin=c_min, vmax=c_max)
    #xx,yy = np.meshgrid(lons_all,lats_all)
    #im = ax.scatter(xx_0,yy_0,c=vs_all, vmin = c_min, vmax=c_max,cmap='gist_rainbow')
    #im = ax[0].pcolormesh(xx.reshape(M,M),yy.reshape(M,M),vs,cmap='gist_rainbow')
    #im = ax[0].pcolormesh(xx.reshape(M,M),yy.reshape(M,M),vs, vmin = c_min, vmax=c_max,cmap='gist_rainbow')
    #plt.colorbar(im)
    #for i in range(len(faults)):
    #    ax[0].plot(faults['clark'+str(i+1)]['lon'], faults['clark'+str(i+1)]['lat'], 'k')


    #for i in range(len(faults)):
    #    ax[1].plot(faults['clark'+str(i+1)]['lon'], faults['clark'+str(i+1)]['lat'], 'k')

    #ax.plot(lons_coyote, lats_coyote, 'k')
    filename = dir_image + 'Vs_hori_profile_'+str((i+1)*dz)+'m_single.png'
    plt.savefig(filename,dpi=100)
    plt.close()